# Import Libraries

In [1]:
from bs4 import BeautifulSoup
import requests

# User Agent

In [2]:
HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

# Scraping

In [3]:
def scrapePage(page):
    review_stars = []
    reviews = []
    #for each card in the page
    page = BeautifulSoup(page.text, "html.parser")
    for card in page.find_all("div",attrs={'data-hook':'review'},class_='a-section review aok-relative'):
        #find star rating
        stari = card.find("i",attrs={'data-hook':'review-star-rating'})
        if stari==None:
          continue
        stars = stari.find("span",class_='a-icon-alt')
        #find review
        review = card.find("span",attrs={'data-hook':'review-body','class':'a-size-base review-text review-text-content'})
        # review = review.find("span")
        if review == None or stars==None:
          continue
        review_stars.append(int(stars.text[0]))
        reviews.append(review.text.strip())
    return reviews, review_stars

In [19]:
from tqdm.notebook import tqdm_notebook

reviews=[]
stars=[]
LIMIT=1500
count200 = 0

url_link='https://www.amazon.in/Juarez-Acoustic-Cutaway-038C-Strings/product-reviews/B017NPCSLI/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber='
i=2
for i in tqdm_notebook(range(1,LIMIT),desc='Scraping:'):
  page = requests.get(url_link + str(i), headers=HEADERS)
  i+=1
  if page.status_code==200:
    count200+=1
    reviewsp, starsp = scrapePage(page)
    reviews.extend(reviewsp)
    stars.extend(starsp)
print("# PAGE STATUS 200: ",count200,"/",LIMIT)


Scraping::   0%|          | 0/1499 [00:00<?, ?it/s]

# PAGE STATUS 200:  1499 / 1500


In [44]:
reviews

["It's the best 🎻 for beginners and the music emanating from this is at par with other costlier 🎻. Thanks for delivering at the given time and the response by the person at the customer care was a pleasant experience.  Thanks  a lot.",
 "Good product. Good finishing. Good delivery service. Good packaging. Cost effective. I don't see any negative points as described in some of the reviews.I recommend beginners, go for it. Product even reviewed by Alison Sparrow, a professional violinist and was given a thumbs up (search in YouTube).No problems with the product yet.",
 'Good product for beginners',
 'I am no expert, it is one nice quality product.',
 "This is the worst pieces of violin and the buying this violin means throwing my money in the dustbin , the bow of this violin is of worst quantity and getting loose day by day.Pls don't waste your money in buying this junck",
 'Very good finishing but sound quality is not atall melodious. Very much metalic sound. I purchased but i had to re

In [20]:
print(len(stars),len(reviews))

4990 4990


# Saving in .csv

In [9]:
import pandas as pd

In [21]:
df = pd.DataFrame()
df['reviews']=reviews
df['stars']=stars

In [22]:
df.head()

,reviews,stars
0,Pretty decent guitar for beginners... Best for...,4
1,Very nice👍👍👍👍,4
2,Awesome,4
3,"I haven't received strap, steings and 2 picks",4
4,I like the feature,4


In [12]:
df.stars.unique()

array([4, 5, 3, 1, 2])

In [23]:
df.shape

(4990, 2)

In [25]:
df = df.drop_duplicates()
df

,reviews,stars
0,Pretty decent guitar for beginners... Best for...,4
1,Very nice👍👍👍👍,4
2,Awesome,4
3,"I haven't received strap, steings and 2 picks",4
4,I like the feature,4
...,...,...
4985,Don't think to much if you are a beginner then...,5
4986,"Very average guitar, don't expect too much,u c...",1
4987,Awesome product great quality ...Awesome sound...,5
4988,Totally DisappointedIt was nothing as shown in...,1


In [26]:
df['stars'].value_counts()

5    1791
4    1098
1     780
3     511
2     248
Name: stars, dtype: int64

In [33]:
df.describe()

,stars
count,4428.000000
mean,3.648600
std,1.485357
min,1.000000
25%,3.000000
50%,4.000000
75%,5.000000
max,5.000000


In [28]:
df.to_csv('./JUAREZ_Acoustic_Guitar_amazon_reviews_raw.csv',index=False)